In [3]:
def parse_input(filename):
    with open(filename, 'r') as f:
        data = f.read()
        data = data.split('\n\n')
        #data = data.replace('\n','')
    data = [d.split('\n') for d in data]
    rules_dict = {}
    
    # process rules
    for r in data[0]:
        split = r.split(': ')
        rule_name = split[0]
        rules = [a.split('-') for a in split[1].split(' or ')]
        min1 = int(rules[0][0])
        max1 = int(rules[0][1])
        min2 = int(rules[1][0])
        max2 = int(rules[1][1])
        rules_dict[rule_name] = {'min1':min1, 'max1':max1, 'min2':min2, 'max2':max2}
    
    # create your ticket
    your_ticket = [int(n) for n in data[1][1].split(',')]
    
    # create nearby tickets
    nearby_tickets = []
    for t in data[2][1:]:
        nearby_tickets.append([int(a) for a in t.split(',')])
    
    return(rules_dict, your_ticket, nearby_tickets)

def check_ticket(ticket, rules):
    error_rate = 0
    error_count = 0
    ticket_length = len(ticket)
    for i in range(ticket_length):
        valid_fields = []
        num = ticket[i]
        valid = False
        for r in rules:
            inRange1 = True if (rules[r]['min1'] <= num and num <= rules[r]['max1']) else False
            inRange2 = True if (rules[r]['min2'] <= num and num <= rules[r]['max2']) else False
            if inRange1 or inRange2:
                valid = True
        if not valid:
            error_rate += num
            error_count += 1
    return(error_rate, error_count)

def check_ticket_list(ticket_list, rules):
    total_error = 0
    for t in ticket_list:
        error, error_count = check_ticket(t, rules)
        total_error += error
    return(total_error)

def get_good_tickets(ticket_list, rules):
    good_tickets = []
    for t in ticket_list:
        error, error_count = check_ticket(t, rules)
        if error_count == 0:
            good_tickets.append(t)
    return(good_tickets)

def create_valid_rules_lists(tickets, rules):
    rule_count = len(rules)
    rule_list = []
    rule_valid_list = []
    for r in rules:
        valid_list = []
        rule_list.append(r)
        for i in range(rule_count):
            valid_rule = True
            ticket_count = 0
            for t in tickets:
                ticket_count += 1
                num = t[i]
                inRange1 = True if (rules[r]['min1'] <= num and num <= rules[r]['max1']) else False
                inRange2 = True if (rules[r]['min2'] <= num and num <= rules[r]['max2']) else False
                if not (inRange1 or inRange2):
                    valid_rule = False
            if valid_rule:
                valid_list.append(i)
        rule_valid_list.append(valid_list)
    return(rule_list, rule_valid_list)

def full_reduce(rvli):
    if max([len(row) for row in rvli]) == 1:
        return
    else:
        for r in rvli:
            if len(r) == 1:
                remove_val = r[0]
                for l in rvli:
                    if len(l) > 1 and remove_val in l:
                        l.remove(remove_val)
        full_reduce(rvli)

def get_answer(ticket, qualifier, rl, rvl):
    result = 1
    for i in range(len(rl)):
        if rl[i].startswith(qualifier):
            loc = rvl[i][0]
            val = your_ticket[loc]
            result = result * val
    return(result)

rules_dict, your_ticket, nearby_tickets = parse_input('input_day16.txt')
print('Part 1 Answer:',check_ticket_list(nearby_tickets, rules_dict))

good_tickets = get_good_tickets(nearby_tickets, rules_dict)
rule_list, rule_valid_list = create_valid_rules_lists(good_tickets, rules_dict)
full_reduce(rule_valid_list)
print('Part 2 Answer:',get_answer(your_ticket, 'departure', rule_list, rule_valid_list))


Part 1 Answer: 28884
Part 2 Answer: 1001849322119
